<a href="https://colab.research.google.com/github/Ashraf1292/Thesis-Fake_jobs-/blob/main/Faisal_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Fixed Bengali Hate Speech Detection Code

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import evaluate
import torch
import warnings
import os

# Disable warnings and wandb
warnings.filterwarnings('ignore')
os.environ['WANDB_DISABLED'] = 'true'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Load dataset
df = pd.read_csv('/content/Bengali hate speech .csv')

# Check actual column names
print("Column names:", df.columns.tolist())
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

# Fix column names based on your actual dataset
TEXT_COLUMN = 'sentence'  # Change this if your text column has a different name
LABEL_COLUMN = 'hate'     # Change this if your label column has a different name

# Drop rows with missing values
df = df.dropna(subset=[TEXT_COLUMN, LABEL_COLUMN])
print(f"\nDataset after cleaning: {len(df)} samples")

# Create binary labels (0: non-hate, 1: hate)
df['binary_label'] = df[LABEL_COLUMN].apply(lambda x: 0 if x == 0 else 1)

# Show class distribution
print("\nBinary class distribution:")
print(df['binary_label'].value_counts())
print("\nOriginal label distribution:")
print(df[LABEL_COLUMN].value_counts())

# Binary classification dataset
binary_df = df[[TEXT_COLUMN, 'binary_label']].copy()

# Multiclass classification: only hate samples (label != 0)
hate_samples = df[df['binary_label'] == 1].copy()

# Map original labels to 0-indexed for multiclass
unique_hate_labels = sorted(hate_samples[LABEL_COLUMN].unique())
label_to_idx = {label: idx for idx, label in enumerate(unique_hate_labels)}
hate_samples['category'] = hate_samples[LABEL_COLUMN].map(label_to_idx)
multi_df = hate_samples[[TEXT_COLUMN, 'category']].copy()

print(f"\nMulticlass samples: {len(multi_df)}")
print("Category distribution:")
print(multi_df['category'].value_counts())
print("Label mapping:", label_to_idx)

# Choose model
MODEL_NAME = 'sagorsarker/bangla-bert-base'  # Or try 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

#=============================================================================
# STEP 1: BINARY CLASSIFICATION (Hate vs Non-Hate)
#=============================================================================

print("\n" + "="*50)
print("STEP 1: BINARY CLASSIFICATION")
print("="*50)

# Split binary data
train_df, test_df = train_test_split(
    binary_df,
    test_size=0.2,
    stratify=binary_df['binary_label'],
    random_state=42
)

print(f"Binary train samples: {len(train_df)}")
print(f"Binary test samples: {len(test_df)}")

# Convert to HuggingFace datasets
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

def preprocess_binary(examples):
    # Tokenize the text
    tokenized = tokenizer(
        examples[TEXT_COLUMN],
        truncation=True,
        padding='max_length',
        max_length=128  # Adjust as needed
    )
    # Add labels and cast to float
    tokenized['labels'] = [float(label) for label in examples['binary_label']]
    return tokenized

# Apply preprocessing
train_ds = train_ds.map(preprocess_binary, batched=True)
test_ds = test_ds.map(preprocess_binary, batched=True)

# Set format for PyTorch
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Binary model
binary_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Suppress the specific warning about newly initialized weights
print("Note: New classifier layers are expected and will be trained.")

# Metrics
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Training arguments - Fixed with processing_class parameter
training_args = TrainingArguments(
    output_dir="./binary_output",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=2e-5,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=100,
    warmup_steps=100,
    fp16=True,
    report_to=None,  # Disable wandb reporting
    run_name="binary_hate_detection"  # Set explicit run name
)

# Trainer - Using processing_class instead of tokenizer
trainer = Trainer(
    model=binary_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    processing_class=tokenizer,  # Changed from tokenizer to processing_class
    compute_metrics=compute_metrics,
)

# Train binary model
print("Training binary model...")
trainer.train()

# Evaluate binary model
print("\nEvaluating binary model...")
binary_results = trainer.evaluate()
print("Binary model results:", binary_results)

# Get predictions on test set
binary_preds = trainer.predict(test_ds)
pred_labels = np.argmax(binary_preds.predictions, axis=-1)
pred_probs = torch.softmax(torch.tensor(binary_preds.predictions), dim=-1)
max_probs = torch.max(pred_probs, dim=-1)[0]

# Add predictions to test dataframe
test_df = test_df.reset_index(drop=True)
test_df['predicted_hate'] = pred_labels
test_df['confidence'] = max_probs.numpy()

print(f"\nBinary classification accuracy: {accuracy.compute(predictions=pred_labels, references=test_df['binary_label'])['accuracy']:.4f}")




Column names: ['sentence', 'hate', 'category']
Dataset shape: (30000, 3)

First few rows:
                                            sentence  hate category
0                     যত্তসব পাপন শালার ফাজলামী!!!!!     1   sports
1                  পাপন শালা রে রিমান্ডে নেওয়া দরকার     1   sports
2  জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...     1   sports
3                শালা লুচ্চা দেখতে পাঠার মত দেখা যায়     1   sports
4   তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব     1   sports

Dataset after cleaning: 30000 samples

Binary class distribution:
binary_label
0    20000
1    10000
Name: count, dtype: int64

Original label distribution:
hate
0    20000
1    10000
Name: count, dtype: int64

Multiclass samples: 10000
Category distribution:
category
0    10000
Name: count, dtype: int64
Label mapping: {np.int64(1): 0}

STEP 1: BINARY CLASSIFICATION
Binary train samples: 24000
Binary test samples: 6000


Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Note: New classifier layers are expected and will be trained.


...

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Number of hate categories: 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Training multiclass model...


RuntimeError: Found dtype Long but expected Float

In [3]:
#=============================================================================
# STEP 2: MULTICLASS CLASSIFICATION (Hate Categories)
#=============================================================================

print("\n" + "="*50)
print("STEP 2: MULTICLASS CLASSIFICATION")
print("="*50)

if len(multi_df) < 100:
    print("Warning: Very few hate samples for multiclass training!")

# Split multiclass data
train_df_multi, test_df_multi = train_test_split(
    multi_df,
    test_size=0.2,
    stratify=multi_df['category'],
    random_state=42
)

print(f"Multiclass train samples: {len(train_df_multi)}")
print(f"Multiclass test samples: {len(test_df_multi)}")

# Convert to HuggingFace datasets
train_ds_multi = Dataset.from_pandas(train_df_multi)
test_ds_multi = Dataset.from_pandas(test_df_multi)

def preprocess_multi(examples):
    tokenized = tokenizer(
        examples[TEXT_COLUMN],
        truncation=True,
        padding='max_length',
        max_length=128
    )
    # Add labels and cast to float
    tokenized['labels'] = [float(label) for label in examples['category']]
    return tokenized

# Apply preprocessing
train_ds_multi = train_ds_multi.map(preprocess_multi, batched=True)
test_ds_multi = test_ds_multi.map(preprocess_multi, batched=True)

# Set format for PyTorch
train_ds_multi.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_ds_multi.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Multiclass model
num_classes = multi_df['category'].nunique()
print(f"Number of hate categories: {num_classes}")

multi_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_classes)

# Training arguments for multiclass
training_args_multi = TrainingArguments(
    output_dir="./multi_output",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,  # More epochs for multiclass
    weight_decay=0.01,
    learning_rate=2e-5,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    warmup_steps=100,
    report_to=None,  # Disable wandb reporting
    run_name="multiclass_hate_detection"  # Set explicit run name
)

# Trainer for multiclass
trainer_multi = Trainer(
    model=multi_model,
    args=training_args_multi,
    train_dataset=train_ds_multi,
    eval_dataset=test_ds_multi,
    processing_class=tokenizer,  # Changed from tokenizer to processing_class
    compute_metrics=compute_metrics,
)

# Train multiclass model
print("Training multiclass model...")
trainer_multi.train()

# Evaluate multiclass model
print("\nEvaluating multiclass model...")
multi_results = trainer_multi.evaluate()
print("Multiclass model results:", multi_results)

# Save models
print("\nSaving models...")
trainer.save_model('./final_binary_model')
trainer_multi.save_model('./final_multiclass_model')



STEP 2: MULTICLASS CLASSIFICATION
Multiclass train samples: 8000
Multiclass test samples: 2000


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Number of hate categories: 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Training multiclass model...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.006300,0.001931,1.000000
2,0.001500,0.000184,1.000000
3,0.001100,0.000097,1.000000
4,0.000800,0.000022,1.000000



Evaluating multiclass model...


Multiclass model results: {'eval_loss': 0.001930548227392137, 'eval_accuracy': 1.0, 'eval_runtime': 4.0434, 'eval_samples_per_second': 494.633, 'eval_steps_per_second': 30.915, 'epoch': 4.0}

Saving models...


In [6]:
# =============================================================================
# STEP 3: TWO-STAGE PREDICTION DEMO
# =============================================================================

print("\n" + "="*50)
print("STEP 3: TWO-STAGE PREDICTION DEMO")
print("="*50)

# Function for two-stage prediction
def predict_two_stage(texts, confidence_threshold=0.8):
    """
    Two-stage prediction: Binary first, then multiclass for confident hate predictions
    """
    # Tokenize input
    inputs = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')

    # Determine the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move inputs to the device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Move models to the device if not already there
    binary_model.to(device)
    multi_model.to(device)

    # Stage 1: Binary prediction
    with torch.no_grad():
        binary_outputs = binary_model(**inputs)
        binary_probs = torch.softmax(binary_outputs.logits, dim=-1)
        binary_preds = torch.argmax(binary_probs, dim=-1)
        binary_confidences = torch.max(binary_probs, dim=-1)[0]

    results = []

    # Move binary predictions and confidences back to CPU for processing if on GPU
    binary_preds = binary_preds.cpu()
    binary_confidences = binary_confidences.cpu()


    for i, (pred, conf) in enumerate(zip(binary_preds, binary_confidences)):
        if pred == 0:  # Non-hate
            results.append({
                'text': texts[i][:50] + "...",
                'binary_pred': 'Non-Hate',
                'multiclass_pred': 'N/A',
                'confidence': conf.item()
            })
        elif conf >= confidence_threshold:  # High-confidence hate
            # Stage 2: Multiclass prediction
            single_input = {k: v[i:i+1].to(device) for k, v in inputs.items()} # Ensure single input is on device
            with torch.no_grad():
                multi_outputs = multi_model(**single_input)
                multi_probs = torch.softmax(multi_outputs.logits, dim=-1)
                multi_pred = torch.argmax(multi_probs, dim=-1)
                multi_conf = torch.max(multi_probs, dim=-1)[0]

            # Move multiclass prediction and confidence back to CPU
            multi_pred = multi_pred.cpu()
            multi_conf = multi_conf.cpu()


            # Map back to original label
            # Check if unique_hate_labels is not empty and multi_pred.item() is within bounds
            if unique_hate_labels and multi_pred.item() < len(unique_hate_labels):
                original_label = unique_hate_labels[multi_pred.item()]
            else:
                original_label = "Unknown Category" # Handle cases where label mapping might be an issue


            results.append({
                'text': texts[i][:50] + "...",
                'binary_pred': 'Hate',
                'multiclass_pred': f'Category {original_label}',
                'confidence': multi_conf.item()
            })
        else:  # Low-confidence hate
            results.append({
                'text': texts[i][:50] + "...",
                'binary_pred': 'Hate (Low Confidence)',
                'multiclass_pred': 'Uncertain',
                'confidence': conf.item()
            })

    return results

# Demo with some test samples
demo_texts = test_df[TEXT_COLUMN].head(10).tolist()
demo_results = predict_two_stage(demo_texts)

print("\nDemo Results:")
for i, result in enumerate(demo_results):
    print(f"\nSample {i+1}:")
    print(f"Text: {result['text']}")
    print(f"Binary: {result['binary_pred']}")
    print(f"Multiclass: {result['multiclass_pred']}")
    print(f"Confidence: {result['confidence']:.4f}")

print("\n✅ Training completed successfully!")
print("Models saved to './final_binary_model' and './final_multiclass_model'")


STEP 3: TWO-STAGE PREDICTION DEMO

Demo Results:

Sample 1:
Text: পয়েতৃশ লাখ টাকা আয় লাখ টাকার ঘড়ি হা হা হা ।...
Binary: Non-Hate
Multiclass: N/A
Confidence: 0.9996

Sample 2:
Text: আগে।সঠিক।তদন।করতে।হবে তার।পর বিচার...
Binary: Non-Hate
Multiclass: N/A
Confidence: 0.9993

Sample 3:
Text: গুলো নাস্তিকরা দিছে👿...
Binary: Non-Hate
Multiclass: N/A
Confidence: 0.8230

Sample 4:
Text: খানকিমাগি মিন্নি চুতমারানি মারাইছে ওর লাং দিয়া ছেল...
Binary: Hate
Multiclass: Category 1
Confidence: 1.0000

Sample 5:
Text: অসাদরুন, মামুন ভাই,এবং মাসুম ভাই, নতুন গুলপ নতুন র...
Binary: Non-Hate
Multiclass: N/A
Confidence: 0.9995

Sample 6:
Text: সাকিব অপু চাই...
Binary: Non-Hate
Multiclass: N/A
Confidence: 0.9995

Sample 7:
Text: প্রত্যেক টা থাপ্পড় কলিজায় লাগছে ওরে পাইলে আনলিমিট...
Binary: Non-Hate
Multiclass: N/A
Confidence: 0.9946

Sample 8:
Text: ভাই আপনাদে কাছে অনুরুদ যদি দয়া হয় তাহলে আজকে শাবান...
Binary: Non-Hate
Multiclass: N/A
Confidence: 0.9994

Sample 9:
Text: স্যারের কোনো দোষ নাই সব দোষ তাহেরির.